---
<div align="center">
    <h2> CRIADOR DE TABELAS PostgreSQL NA AZURE
    </h2>
</div>

---

<div aligh="justify">
    <p> O código a seguir faz uso das queries em SQL disponibilizada pela <a ref="https://github.com/julianazanelatto/power_bi_analyst/tree/b010c1874a183a0fb4b831e76dc68b9b872becec/M%C3%B3dulo%203/Desafio%20de%20Projeto">DIO</a> para criar e popular as bases de dados necessárias em um banco de dados na AZURE. Naturalmente, a fim de alinhar com minha prática profissional, adequei o problema, originalmente para MySQL na AZURE para PostgreSQL na AZURE.
    </p>
</div>

---

In [1]:
# Bibliotecas necessárias para o presente projeto
import psycopg2
import re
import pandas as pd
from unidecode import unidecode
from os import listdir

In [2]:
# Arquivo de credenciais necessárias para acessar banco de dados
credentials = pd.read_csv("../credentials/credentials.csv").iloc[0].to_dict()

In [3]:
# Função para criar conexão e cursor
def conectar():
    host = credentials['host']
    dbname = credentials['database']
    user = credentials['user']
    password = credentials['password']
    sslmode = "require"

    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
    conexao = psycopg2.connect(conn_string) 
    print("Connection established")
    cursor = conexao.cursor()

    cursor = conexao.cursor()
    return conexao, cursor

# Função para executar uma query
def executar_query(cursor, query):
    cursor.execute(query)
    # Verificar se a consulta retorna resultados (SELECT)
    if cursor.description is not None:
        colunas = [desc[0] for desc in cursor.description]
        resultados = cursor.fetchall()
        return pd.DataFrame(resultados, columns=colunas)
    else:
        # Se não há resultados, retornar None ou outra indicação apropriada
        return None

# Função para fechar a conexão e o cursor
def fechar_conexao(conexao, cursor):
    cursor.close()
    conexao.close()
    print("Connection finished")

---

In [4]:
queries_sql = [ x for x in listdir("../queries/") if x.__contains__(".sql")]

In [5]:
# Especifique o caminho do arquivo SQL
caminho_arquivo_sql = '../queries/'
queries = {}
for query in queries_sql:

    # Atribuindo queries às 
    with open(caminho_arquivo_sql+query, 'r') as arquivo_sql:
        conteudo_sql = arquivo_sql.read()
    queries[query[:-4]] = conteudo_sql

---

In [7]:
## Cria conexão
conexao, cursor = conectar()

for query in queries.values():
    # Executa a query
    executar_query(cursor, query)

# Comita possível criação de nova tabela
conexao.commit()

# Fecha conexão
fechar_conexao(conexao, cursor)

Connection established
Connection finished
